##  Programmation Orientée Objet et Gestion de fichiers

> Tutoriel complet illustrant les concepts de la programmation orientée objet

> Système de gestion bancaire

In [7]:
"""
PARTIE 1 : Classes et objets : Permet de modéliser des entités réelles (ici un compte bancaire)
"""
class Client:
    # Représente un client bancaire
    def __init__(self, nom: str, email: str):
        self.nom, self.email = nom, email
    
    def __str__(self) -> str:
        return f"Client: {self.nom} ({self.email})"
    
"""
PARTIE 2 : Attributs et méthodes
"""
class CompteBancaire:
    # Représente un compte bancaire
    taux_interet: float = 0.02 # attruibut de classe
    
    def __init__(self, client: Client, solde_initial: float = 0.0):
        self.client, self.solde = client, solde_initial # attribut d'instance (client)
    
    def deposer(self, montant: float) -> None:
        self.solde = self.solde + montant
     
    def retirer(self, montant: float) -> None:
        if montant > self.solde:
            raise ValueError("Fonds inssuffisants")
        self.solde = self.solde - montant
        
    @classmethod
    def changer_taux_interet(cls, nouveau_taux: float) -> None:
        cls.taux_interet = nouveau_taux
        
    @staticmethod
    def convertir_euro_usd(montant: float, taux: float = 1.1) -> float:
        return montant * taux
    
"""
PARTIE 3 : Encapsulation : Protection des données via les getters et les setters
"""
class CompteSecure:
    def __init__(self, solde_initial: float = 0.0):
        self._solde = solde_initial # convention protected
    # pythonic way with property decorator    
    @property
    def solde(self) -> float:
        return self._solde
    
    @solde.setter
    def solde(self, valeur: float) -> None:
        if valeur < 0:
            raise ValueError("Le solde ne peut pas etre négatif")
        self._solde = valeur
        
    # non pythonic way with explicit getter and setter
    def get_solde(self) -> float:
        return self._solde
    def set_solde(self, valeur: float) -> None:
        if valeur < 0:
            raise ValueError("Le solde ne peut pas etre négatif")
        self._solde = valeur
    
"""
PARTIE 4 : Héritage
"""

class CompteEpargne(CompteBancaire):
    def appliquer_interet(self):
        self.solde = self.solde + (self.solde * self.taux_interet)
    def __str__(self):
        return f"Compte Epargne de {self.client.nom}: {self.solde:.2f}€"
    
    
"""
PARTIE 5 : Polymorphisme : Meme méthode, comportement différent
"""
class CompteCourant(CompteBancaire):
    def __str__(self) -> str:
        return f"Compte Courant de {self.client.nom}: {self.solde:.2f}€"
    
"""
PARTIE 6 : Abstraction
"""
from abc import ABC, abstractclassmethod

class Transaction(ABC):
    @abstractclassmethod
    def executer(self, compte: CompteBancaire) -> None:
        pass
    
class Depot(Transaction):
    def __init__(self, montant: float):
        self.montant = montant

    def executer(self, compte: CompteBancaire) -> None:
        compte.deposer(self.montant)

class Retrait(Transaction):
    def __init__(self, montant: float):
        self.montant = montant

    def executer(self, compte: CompteBancaire) -> None:
        compte.retirer(self.montant)
    
    
# Démonstration en utilisant tous les concepts
# création d'un client et comptes
client = Client("Sammy", "sammy@example.com")
c_courant = CompteCourant(client, 5000)
c_epargne = CompteEpargne(client, 500)

print(c_courant)
print(c_epargne)

# Transaction avec abs
depot = Depot(350)
depot.executer(c_courant)
retrait = Retrait(20)
retrait.executer(c_epargne)

# changement du taux d'intérêt
CompteBancaire.changer_taux_interet(0.03) # en utilisant la méthode de classe
c_epargne.appliquer_interet()
print("Après transactions et application des intérêts:")
print(c_courant)
print(c_epargne)

# appel de __str__ pour démontrer le polymorphism
comptes = [c_courant, c_epargne]
for c in comptes:
    print(c)
    
# utilisation de la méthode statique
print(f"100€ en USD ($){CompteBancaire.convertir_euro_usd(100):.2f}")

# utilisation des getters et setters avec encapsulation, @property
# @property permet transformer une méthode en attribut, c'est à dire qu'on peut l'utiliser sans les parenthèses lorsque on l'appelle et on peut aussi lui attribuer une valeur lorsqu'on utilise le setter (lorsque le fonction prend quelque chose en argument)
compte_secure = CompteSecure(100)
print(f"Solde initial: {compte_secure.solde}€")
compte_secure.solde = 200
print(f"Nouveau solde: {compte_secure.solde}€")

Compte Courant de Sammy: 5000.00€
Compte Epargne de Sammy: 500.00€
Après transactions et application des intérêts:
Compte Courant de Sammy: 5350.00€
Compte Epargne de Sammy: 494.40€
Compte Courant de Sammy: 5350.00€
Compte Epargne de Sammy: 494.40€
100€ en USD ($)110.00
Solde initial: 100€
Nouveau solde: 200€


## @property

In [3]:
# Getter simple (lecture dynamique)

class Rectangle:
    def __init__(self, largeur: float, hauteur: float):
        self.largeur, self.hauteur = largeur, hauteur
    @property
    def aire(self):
        return self.largeur * self.hauteur
    
# Utilisation
r = Rectangle(4,6)
print(r.aire) # 24, pas besoin d'écire r.aire()

24


In [4]:
# Getter + Setter

class Compte:
    def __init__(self, solde):
        self._solde = solde  # attribut protégé

    @property
    def solde(self):
        """Getter : lire le solde"""
        return self._solde

    @solde.setter
    def solde(self, nouveau_solde):
        """Setter : modifier avec validation"""
        if nouveau_solde < 0:
            raise ValueError("Le solde ne peut pas être négatif")
        self._solde = nouveau_solde


# --- Utilisation ---
c = Compte(100)
print(c.solde)   # 100 → passe par le getter
c.solde = 200    # OK → passe par le setter
# c.solde = -50  #  Erreur

100


In [ ]:
# Getter + setter + Deleter (Pour controler la supperession d'un attribut)
class Employe:
    def __init__(self, nom: str):
        self._nom = nom
        
    @property
    def nom(self):
        return self._nom
    
    @nom.setter
    def nom(self, nouveau_nom):
        if not nouveau_nom.strip():
            raise ValueError("Nom invlide")
        self._nom = nouveau_nom
    
    @nom.deleter
    def nom(self):
        print("Suppression du nom...")
        del self._nom
# Utilisation
e = Employe("Emy")
print(e.nom) # getter
e.nom = "Sam" # setter
del e.nom       #deleter

In [5]:
class Cercle:
    def __init__(self, rayon):
        self.rayon = rayon

    @property
    def diametre(self):
        return self.rayon * 2

    @property
    def aire(self):
        import math
        return math.pi * (self.rayon ** 2)


# --- Utilisation ---
c = Cercle(10)
print(c.diametre)  # 20
print(c.aire)      # 314.15...
c.rayon = 5
print(c.aire)      # 78.5... (recalculé automatiquement)

20
314.1592653589793
78.53981633974483
